After creating the Alphabet, we are create the full words that represents the ECG

In [1]:
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
from collections import Counter
import pandas as pd
import numpy as np
from time import time
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
np.set_printoptions(suppress=True,precision=10)

In [2]:
def obtainSubjects(dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("SELECT DISTINCT subject_id "
                      " FROM waveformqrst WHERE centroid IS NOT NULL" # limit 150"
    )
    cur.execute(select_stament)
    subject = []
    for row in cur :
        subject.append(row[0])
    conn.close()
    return subject

In [3]:
def obtainWord(subject,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("SELECT centroid "
                      " FROM waveformqrst WHERE subject_id="+str(subject)+" ORDER BY qrtsorder"
    )
    cur.execute(select_stament)
    centroids = ""
    for row in cur :
        centroid = row[0]
        if centroid is not None :
            centroids= centroids+centroid
    conn.close()
    return centroids

In [4]:
def deleteWord(dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "DELETE FROM subjectwords"
    cur.execute(select_stament)
    conn.commit()
    cur.close()
    conn.close()

In [5]:
def instertWord(words,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    insert_statement = 'INSERT into subjectwords (%s) values %s'
    columns = words.keys()
    values = [words[column] for column in columns]
#    print(cur.mogrify(insert_statement, (AsIs(','.join(columns)), tuple(values))))
    cur.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
    conn.commit()
    cur.close()
    conn.close()

In [6]:
def selectWord(dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("SELECT subject_id,word "
                      " FROM subjectwords WHERE length(word)>1000"
    )
    cur.execute(select_stament)
    select = []
    for row in cur :
        select.append([row[0],row[1]])
    conn.close()
    return select

In [7]:
deleteWord()
subjects = obtainSubjects() 
print(len(subjects))
for subject in subjects :
    word = obtainWord(subject)
    if word is not None:
        words = {'subject_id':subject,'word':word}
        print(len(word),end=":")
        instertWord(words)
        print(words['subject_id'],end=",")

366
3316:3623,19077:16639,1996:7153,83:8985,12695:9070,1474:6901,12801:7009,20229:7442,18536:7492,12464:12217,13516:571,8284:9732,14811:7612,14091:6673,3232:10487,2512:6889,5492:8814,13554:2990,18321:8698,18758:5933,16256:8533,8747:10656,8850:10510,14707:10738,492:7521,10691:3612,4878:8573,9711:2442,21960:8347,12944:1908,13659:11931,7445:7452,2607:3554,19713:4059,1339:7360,19590:4317,17769:6981,16256:20,18828:2148,14355:4068,18616:10595,19579:8198,7236:10209,16901:3745,19691:9425,3950:7522,17238:5336,7210:16172,53:8996,14527:3622,718:7981,7681:9335,18999:8415,25:90495,1165:1802,7:7517,17212:2185,350:17218,10722:10534,62:2228,15762:10766,12702:8009,4408:8936,10929:8087,31:16909,16902:12251,21314:12215,10005:9871,15761:10096,9622:8670,20614:7651,20151:9354,6974:9672,4497:6708,16262:9705,14755:10186,7280:5766,9684:7084,8754:7528,15553:7172,18395:135,4315:16709,7305:9630,1716:7799,6180:4865,14987:7685,15688:7289,2918:17182,20034:7860,10783:16360,20312:9366,13916:4180,10881:8336,985:11951,2

In [8]:
def get_all_substrings(input_string,length=5):
    substrings = []
    for j in range(len(input_string)) :
        for i in range(length) :
            substrings.append(input_string[j:j+i+1])
    return Counter(substrings)

In [9]:
def existMatrix(word,subject,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("SELECT 1 "
                      " FROM matrix WHERE subject_id='"+str(subject)+"' AND word='"+str(word)+"'"
    )
    cur.execute(select_stament)
    exist = False
    for row in cur :
        exist = True
    cur.close()
    conn.close()
    return exist

In [10]:
def saveMatrix(matrix,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    insert_statement=('INSERT INTO matrix(word,subject_id,counting)'
                      ' SELECT unnest( %(word)s ) ,'
                      ' unnest( %(subject_id)s) ,'
                      ' unnest( %(counting)s)')
    word=[r['word'] for r in matrix]
    subject_id=[r['subject_id'] for r in matrix]
    counting=[r['counting'] for r in matrix]
#    print(cur.mogrify(insert_statement,locals()))
    cur.execute(insert_statement,locals())
    conn.commit()
    cur.close()
    conn.close()

In [11]:
def cleanMatrix(dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("DELETE "
                      " FROM matrix"
    )
    cur.execute(select_stament)
    conn.commit()
    cur.close()
    conn.close()

In [12]:
words = selectWord()
cleanMatrix()
i=0
for word in words :
    subject = word[0]
    subs =get_all_substrings(word[1],length=10)
    matrix = []
    for key in subs:
        matrix.append({'word':key,'counting':subs[key],'subject_id':subject})
#        if not existMatrix(key,subject) :
    if matrix != [] :
        print(i,end=",")
        i=i+1
        saveMatrix(matrix)
#    print(subs.keys())

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,27

In [13]:
def selectMatrix(dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("SELECT subject_id,word,counting "
                      " FROM matrix ORDER BY subject_id"
    )
    cur.execute(select_stament)
    select = []
    for row in cur :
        select.append((row))
    cur.close()
    conn.close()
    return select

In [14]:
labels = ['subject_id', 'Word', 'Counting']
df = pd.DataFrame.from_records(selectMatrix(), columns=labels)
table = pd.pivot_table(df,index=["subject_id"],columns=["Word"],values=["Counting"],
                       aggfunc={"Counting":[np.sum]},fill_value=0)
print(table)

           Counting                                                      \
                sum                                                       
Word              a     aa    aaa   aaaa  aaaaa aaaaaa aaaaaaa aaaaaaaa   
subject_id                                                                
20            16265  16263  16261  16259  16257  16255   16253    16251   
33             3038   3031   3024   3017   3010   3003    2996     2989   
85             2821   2319   1941   1651   1426   1241    1085      957   
135           18404  18402  18400  18398  18396  18394   18392    18390   
263           12308  12305  12302  12299  12296  12293   12290    12287   
283           18065  18051  18037  18024  18011  17998   17986    17974   
317            5189   5182   5175   5168   5162   5156    5150     5144   
328            4026   3988   3952   3918   3888   3859    3832     3806   
377           10644  10636  10628  10621  10614  10607   10600    10593   
408            9352   934

In [15]:
# Fit the NMF model
t0 = time()
nmf = NMF(n_components=30, random_state=1,alpha=.1, l1_ratio=.5)
W = nmf.fit_transform(table)
H = nmf.components_
print(W)
print("done in %0.3fs." % (time() - t0))

[[ 59.7198433777   0.             0.           ...,   0.             0.
    0.          ]
 [ 10.9425792887   0.2777407337   0.0001579825 ...,   0.0019648461   0.
    0.          ]
 [  2.738067762   17.8686213274   0.4898717575 ...,   0.             0.
    0.2518760099]
 ..., 
 [ 51.8583587917   2.7670185946   0.0019755654 ...,   0.9659574204
    0.0096008726   0.          ]
 [ 39.0660364907   1.2347535354   0.007121405  ...,   0.2243629173   0.
    0.          ]
 [  2.7443450367  23.4069701784   6.4158242254 ...,   0.4552525055
    5.5139710128   0.          ]]
done in 509.078s.


In [16]:
print(np.shape(W))
print(np.shape(H))

(327, 30)
(30, 136291)


In [17]:
def patientIsAlive(patient,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = ("SELECT dod "
                      " FROM patients WHERE subject_id = "+str(patient)
    )
    cur.execute(select_stament)
    select = []
    for row in cur :
        select.append(1 if row[0] is not None else 0 )
    cur.close()
    conn.close()
    return select

In [18]:
patients = []
for patient in table.index :
    patients.append(patientIsAlive(patient)[0])
print(len(patients))

327


In [19]:
# flatten y into a 1-D array
y = np.ravel(patients)
modelo_lr = LogisticRegression()
modelo_lr.fit(y=y,X=W)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
modelo_lr.score(W, y)

0.58409785932721714

In [21]:
y.mean()

0.54434250764525993

In [25]:
predicion = modelo_lr.predict(W)
print(W[:1])
print(predicion,y[:1])

[[ 59.7198433777   0.             0.             0.             0.             0.
    0.             0.             0.             0.             0.             0.
    0.             0.             0.             0.             0.             0.
    0.             0.             0.             0.             0.
    0.0586727148   0.3381246636   0.             0.             0.             0.
    0.          ]]
[1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 

In [23]:
modelo_lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
y

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1,